# 2D Maxima with Plane Sweep — Student Notebook

**Goal:** Implement the O(n log n) sweep-line algorithm for 2D maxima, compare against brute force, and analyze performance.

## Part A — Manual Warm-up
Given $P=\{(2,5),(4,1),(1,4),(3,3),(5,2),(6,6),(7,4),(5,5)\}$, identify maximal points by inspection (do this on paper), then verify using code below.

In [ ]:
import random, time, math
import numpy as np
import matplotlib.pyplot as plt

P_example = [(2,5),(4,1),(1,4),(3,3),(5,2),(6,6),(7,4),(5,5)]
P_example

## Part B — Sweep-line Implementation

In [ ]:
def maxima_sweep(points):
    """Return list of maximal points using plane sweep (x-desc, track max y)."""
    pts = sorted(points, key=lambda p: (-p[0], -p[1]))  # sort by x desc, y desc as tie-breaker
    maxima = []
    maxY = float('-inf')
    for x, y in pts:
        if y >= maxY:
            maxima.append((x, y))
            maxY = y
    return maxima

# Quick test on the example
maxima_sweep(P_example)

## Part C — Brute-force Verifier

In [ ]:
def dominates(a, b):
    return (a[0] >= b[0]) and (a[1] >= b[1]) and (a != b)

def maxima_bruteforce(points):
    res = []
    for p in points:
        if not any(dominates(q, p) for q in points):
            res.append(p)
    return res

maxima_bruteforce(P_example)

## Part D — Visualization (Optional)

In [ ]:
def plot_points_and_maxima(points, maxima):
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    plt.figure()
    plt.scatter(xs, ys)
    mx = [p[0] for p in maxima]
    my = [p[1] for p in maxima]
    plt.scatter(mx, my, marker='x')
    for p in points:
        plt.text(p[0]+0.05, p[1]+0.05, str(p))
    plt.xlabel('x'); plt.ylabel('y'); plt.title('Points and Maxima')
    plt.show()

plot_points_and_maxima(P_example, maxima_sweep(P_example))

## Part E — Benchmarking
Generate random datasets for $n \in \{10^3, 10^4, 10^5\}$ and compare runtime of sweep vs brute force. (You may reduce $n$ if your runtime budget is limited.)

In [ ]:
def rand_points(n, low=0, high=1_000_000, seed=0):
    rnd = random.Random(seed)
    return [(rnd.randint(low, high), rnd.randint(low, high)) for _ in range(n)]

def time_fn(fn, *args, **kwargs):
    t0 = time.perf_counter()
    out = fn(*args, **kwargs)
    t1 = time.perf_counter()
    return out, (t1 - t0)

sizes = [10**3, 10**4]  # adjust to [10**3, 10**4, 10**5] if resources allow
results = []
for n in sizes:
    pts = rand_points(n, seed=42)
    _, t_sweep = time_fn(maxima_sweep, pts)
    # brute force may be slow for n>=10**4; use with care
    if n <= 5000:
        _, t_brute = time_fn(maxima_bruteforce, pts)
    else:
        t_brute = float('nan')
    results.append((n, t_sweep, t_brute))
results

In [ ]:
# Plot runtimes (log-scale on x-axis)
ns = [r[0] for r in results]
ts_sweep = [r[1] for r in results]
ts_brute = [r[2] for r in results]

plt.figure()
plt.plot(ns, ts_sweep, marker='o', label='Sweep O(n log n)')
plt.plot(ns, ts_brute, marker='o', label='Brute-force O(n^2)')
plt.xscale('log')
plt.xlabel('n (log scale)')
plt.ylabel('seconds')
plt.title('Runtime Comparison')
plt.legend()
plt.show()

## Part F — Discussion
- Why does sorting by descending $x$ make it safe to keep only non-decreasing $y$?\n- What happens if dominance is strict in both coordinates?\n- How would you extend (or why is it hard to extend) to 3D?